In [8]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
from pprint import pprint
import requests
import time
key = '5h567tf39qwsksfp9uxakxfc'

## Source

### https://developer.sportradar.com/

- nfl api

- scores from 2000 - 2017

## Get 17 years of scores form the NFL

- Create a function that inputs:
    - year 
    - season(pre,regular,post)
    - api key

- output:
    - json result

#### Create a loop to create a list for wanted years

In [7]:
years = []
for i in range(2000,2018):
    years.append(str(i))
    
print(years)

['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']


In [36]:
def nfl_schedule(year,season,key):
    '''returns json result from api call'''
    #print('inside func: year:{}:season: {}'.format(year,season))  # debug
    URL = 'https://api.sportradar.us/nfl-ot2/games/'
    path = '{}/{}/schedule.json'.format(year,season)
    payload = {'api_key':key}
    r = requests.get(URL+path,params=payload)
    return r.json()

## how to get  the data

- create an empty list
- loop over our year list
- call our function with our values (year,season, api-key)
- wait 1 second ( api request)
- given that year, get the length and loop over those weeks
- create a dataframe for that given year,and given week
- append that dataframe to the emplty list
- pd.concat our dataframe list


In [37]:
season = 'REG'

df_list = []  # emplty list for our weekly dataframes

for i in years:  # 2000 - 2017
    #print('year:{}'.format(i))  # debug
    json_yr = nfl_schedule(i,season,key)  # call func get json
    time.sleep(1)  # sleep
    for j in range(len(json_yr['weeks'])):  # loop over every week in that year
        #print('year:{},week:{}'.format(i,j))  # debug
        wk_df = json_normalize(json_yr['weeks'][j]['games'])  # df of that week
        df_list.append(wk_df)  # append to empty list
time.sleep(1)  # sleep
nfl_schedule = pd.concat(df_list, axis=0)  # concat list of weekly dataframes

## Inspect our data

In [38]:
nfl_schedule.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4593 entries, 0 to 15
Data columns (total 33 columns):
attendance             4592 non-null float64
away.alias             4593 non-null object
away.game_number       4593 non-null int64
away.id                4593 non-null object
away.name              4593 non-null object
broadcast.internet     81 non-null object
broadcast.network      1536 non-null object
broadcast.satellite    801 non-null object
entry_mode             4593 non-null object
home.alias             4593 non-null object
home.game_number       4593 non-null int64
home.id                4593 non-null object
home.name              4593 non-null object
id                     4593 non-null object
number                 1537 non-null float64
reference              1537 non-null object
scheduled              4593 non-null object
scoring.away_points    4592 non-null float64
scoring.home_points    4592 non-null float64
scoring.periods        4592 non-null object
status          

In [39]:
nfl_schedule.head(5)

,attendance,away.alias,away.game_number,away.id,away.name,broadcast.internet,broadcast.network,broadcast.satellite,entry_mode,home.alias,...,venue.capacity,venue.city,venue.country,venue.id,venue.name,venue.roof_type,venue.state,venue.surface,venue.zip,weather
0,64900.0,DET,1,c5a59daa-53a7-4de0-851f-fb12be893e9e,Detroit Lions,NaN,NaN,NaN,INGEST,NO,...,76468,New Orleans,USA,3c85d89a-ec66-4983-acd5-1381d6c8673a,Mercedes-Benz Superdome,dome,LA,artificial,70112,"Partly Cloudly Temp: 89 F, Wind: West 10 mph"
1,60292.0,TB,1,4254d319-1bc7-4f81-b4ab-b5e6f3402b69,Tampa Bay Buccaneers,NaN,NaN,NaN,INGEST,NE,...,60292,Foxborough,NaN,67412aec-5fa4-4cef-ac07-323b90d9d388,Foxboro Stadium,outdoor,MA,turf,02035,"Cloudy Temp: 71 F, Humidity: 93%, Wind: Variab..."
2,72949.0,SEA,1,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seattle Seahawks,NaN,NaN,NaN,INGEST,MIA,...,76100,Miami Gardens,USA,50a5c833-1570-4c38-abc7-7914cf87dbde,Hard Rock Stadium,outdoor,FL,turf,33056,"Sunny Temp: 89 F, Humidity: 59%, Wind: East 13..."
3,59870.0,NYJ,1,5fee86ae-74ab-4bdd-8416-42a9dd9964f3,New York Jets,NaN,NaN,NaN,INGEST,GB,...,80750,Green Bay,USA,5a60dd3a-302c-41c6-ab0f-dd335c1103c2,Lambeau Field,outdoor,WI,turf,54304,"Mostly cloudy, light winds, high of 77. Temp: ..."
4,80257.0,CAR,1,f14bf5cc-9a82-4a38-bc15-d39f75ed5314,Carolina Panthers,NaN,NaN,NaN,INGEST,WAS,...,83000,Landover,USA,7c11bb2d-4a53-4842-b842-0f1c63ed78e9,FedExField,outdoor,MD,turf,20785,"Temp: 80 F, Humidity: 98%, Wind: Calm mph"


## Save this raw dataframe to csv

In [42]:
ls sports_data_raw/NFL/

nfl10clean.csv*  nfl10yearsfull.csv*


In [43]:
nfl_schedule.to_csv('sports_data_raw/NFL/nfl17yearsfull_raw.csv')

## Slice original dataframe with wanted columns

In [45]:
nfl_df = nfl_schedule[['scheduled','home.alias','scoring.home_points','away.alias','scoring.away_points']]

In [46]:
nfl_df.head()

,scheduled,home.alias,scoring.home_points,away.alias,scoring.away_points
0,2000-09-03T18:03:36+00:00,NO,10.0,DET,14.0
1,2000-09-03T18:04:35+00:00,NE,16.0,TB,21.0
2,2000-09-03T21:15:54+00:00,MIA,23.0,SEA,0.0
3,2000-09-03T21:20:12+00:00,GB,16.0,NYJ,20.0
4,2000-09-03T19:02:58+00:00,WAS,20.0,CAR,17.0


## Inspect columns

In [54]:
nfl_df['scheduled'].unique()

array(['2000-09-03T18:03:36+00:00', '2000-09-03T18:04:35+00:00',
       '2000-09-03T21:15:54+00:00', ..., '2017-12-26T01:30:00+00:00',
       '2017-12-31T18:00:00+00:00', '2017-12-31T21:25:00+00:00'],
      dtype=object)

In [55]:
nfl_df['scheduled'].value_counts(dropna=False).head()

2017-12-31T21:25:00+00:00    9
2017-12-03T18:00:00+00:00    9
2016-09-11T17:00:00+00:00    9
2016-10-16T17:00:00+00:00    9
2017-12-24T18:00:00+00:00    9
Name: scheduled, dtype: int64

In [51]:
nfl_df['home.alias'].unique()

array(['NO', 'NE', 'MIA', 'GB', 'WAS', 'PIT', 'ATL', 'OAK', 'KC', 'NYG',
       'BUF', 'DAL', 'STL', 'MIN', 'CLE', 'SEA', 'TB', 'BAL', 'DET',
       'NYJ', 'PHI', 'ARI', 'SD', 'CIN', 'DEN', 'IND', 'TEN', 'SF', 'CHI',
       'JAC', 'CAR', 'HOU', 'LA', 'LAC'], dtype=object)

In [53]:
nfl_df['home.alias'].value_counts(dropna=False).head()

MIA    145
BAL    144
BUF    144
CIN    144
PIT    144
Name: home.alias, dtype: int64

In [56]:
nfl_df['scoring.home_points'].unique()

array([10., 16., 23., 20.,  0., 36.,  9., 14., 21., 41., 30.,  7., 34.,
       39., 15., 18., 32., 13., 27., 42., 31., 17., 22.,  6., 24., 19.,
       37., 12., 43.,  3., 57., 38., 28., 26., 29., 45., 44., 40., 48.,
       54.,  8., 49., 35., 52., 25., 47., 33., 50., 56., 58., 51.,  5.,
       11., 59., 62., 46., 55.,  2., 53., nan])

In [57]:
nfl_df['scoring.home_points'].value_counts(dropna=False).head()

20.0    339
24.0    302
17.0    285
27.0    281
31.0    244
Name: scoring.home_points, dtype: int64

In [58]:
nfl_df['away.alias'].unique()

array(['DET', 'TB', 'SEA', 'NYJ', 'CAR', 'BAL', 'SF', 'SD', 'IND', 'ARI',
       'TEN', 'PHI', 'DEN', 'CHI', 'JAC', 'STL', 'WAS', 'NE', 'NYG',
       'DAL', 'MIA', 'GB', 'NO', 'CLE', 'ATL', 'OAK', 'KC', 'BUF', 'PIT',
       'MIN', 'CIN', 'HOU', 'LA', 'LAC'], dtype=object)

In [60]:
nfl_df['away.alias'].value_counts(dropna=False).head()

TB     145
BAL    144
BUF    144
CIN    144
PIT    144
Name: away.alias, dtype: int64

In [61]:
nfl_df['scoring.away_points'].unique()

array([14., 21.,  0., 20., 17., 16., 28.,  6., 27., 13., 41., 36., 37.,
       10., 19., 33., 31.,  7., 18., 24., 38.,  3., 15., 23., 29., 12.,
       34.,  9., 35., 30., 11., 26., 48., 45., 22., 25., 42., 32., 40.,
       44., 49., 43., 55., 39.,  5.,  8., 56., 54., 47., 46., 59., 52.,
       51., 50., nan])

In [62]:
nfl_df['scoring.away_points'].value_counts(dropna=False).head()

17.0    351
24.0    304
20.0    300
10.0    288
13.0    266
Name: scoring.away_points, dtype: int64

# Schedule column
- convert to datetime
- set index
- sort index

In [63]:
nfl_df.head()

,scheduled,home.alias,scoring.home_points,away.alias,scoring.away_points
0,2000-09-03T18:03:36+00:00,NO,10.0,DET,14.0
1,2000-09-03T18:04:35+00:00,NE,16.0,TB,21.0
2,2000-09-03T21:15:54+00:00,MIA,23.0,SEA,0.0
3,2000-09-03T21:20:12+00:00,GB,16.0,NYJ,20.0
4,2000-09-03T19:02:58+00:00,WAS,20.0,CAR,17.0


In [64]:
nfl_df['scheduled'] = pd.to_datetime(nfl_df['scheduled'])  # convert to datetime

nfl_df = nfl_df.set_index('scheduled').sort_index(ascending=True) # set and sort index

/home/salas/miniconda3/envs/springboard/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
nfl_df.head()

,home.alias,scoring.home_points,away.alias,scoring.away_points
scheduled,,,,
2000-09-03 00:00:00,PIT,0.0,BAL,16.0
2000-09-03 00:00:00,OAK,9.0,SD,6.0
2000-09-03 18:02:00,ATL,36.0,SF,28.0
2000-09-03 18:03:33,NYG,21.0,ARI,16.0
2000-09-03 18:03:36,NO,10.0,DET,14.0


## Win column

- create colum bases on score from team

pseudocode

```
if value of `scoring.home_points` > `scoring.away_points`
     get value of `home.alias`, 
     else get value of `away.alias`
```

In [66]:
nfl_df['winner'] = np.where(nfl_df['scoring.home_points'] > nfl_df['scoring.away_points'],
                            nfl_df['home.alias'],
                            nfl_df['away.alias'])

In [67]:
nfl_df.head()

,home.alias,scoring.home_points,away.alias,scoring.away_points,winner
scheduled,,,,,
2000-09-03 00:00:00,PIT,0.0,BAL,16.0,BAL
2000-09-03 00:00:00,OAK,9.0,SD,6.0,OAK
2000-09-03 18:02:00,ATL,36.0,SF,28.0,ATL
2000-09-03 18:03:33,NYG,21.0,ARI,16.0,NYG
2000-09-03 18:03:36,NO,10.0,DET,14.0,DET


## Find NaN values

In [68]:
nfl_df['scoring.home_points'].unique()

array([ 0.,  9., 36., 21., 10., 16., 14., 20., 30., 23.,  7., 41., 31.,
       27., 39., 18., 13., 34., 42., 15., 22., 32., 17.,  6., 24., 19.,
       37.,  3., 12., 43., 57., 28., 38., 26., 29., 45., 44., 48., 54.,
       40., 49.,  8., 35., 52., 25., 33., 47., 50., 56., 58., 51.,  5.,
       11., 59., 62., 46., 55.,  2., 53., nan])

In [69]:
nfl_df[nfl_df['scoring.home_points'].isnull()]

,home.alias,scoring.home_points,away.alias,scoring.away_points,winner
scheduled,,,,,
2017-09-10 17:00:00,MIA,NaN,TB,NaN,TB


## Google it!

- it was posponed

[google search](https://www.google.com/search?q=2017-09-10+miami+vs+tampa+bay&rlz=1C1CHBD_enUS769US769&oq=2017-09-10++miami+vs+tampa+bay&aqs=chrome..69i57.12899j0j4&sourceid=chrome&ie=UTF-8#sie=m;/g/11f53rgny4;6;/m/059yj;dt;fp;1)

# Save clean version to csv

In [71]:
ls sports_data_clean/

astros_08_17_clean.csv*       nfl_08_17_clean.csv*  rice_08_17_clean.csv*
dynamo_08_17_clean.csv*       nfl10clean.csv*       tsu_08_17_clean.csv*
hou_rockets_08_17_clean.csv*  nfl10yearsfull.csv*   uh_08_17_clean.csv*


In [72]:
nfl_df.to_csv('sports_data_clean/nfl17years_clean.csv')

## Select only games from 2010 - 2017

In [76]:
nfl_10_17 = nfl_df.loc['2010':'2017']

In [78]:
nfl_10_17.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2065 entries, 2010-01-03 18:02:27 to 2017-12-31 21:25:00
Data columns (total 5 columns):
home.alias             2065 non-null object
scoring.home_points    2064 non-null float64
away.alias             2065 non-null object
scoring.away_points    2064 non-null float64
winner                 2065 non-null object
dtypes: float64(2), object(3)
memory usage: 96.8+ KB


## Select only games from Houston Texans
- create a boolean mask that selects only games where houston was home

In [81]:
houston_home = nfl_10_17['home.alias'] == 'HOU'

In [83]:
hou_home = nfl_10_17[houston_home]

In [85]:
hou_home.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 65 entries, 2010-01-03 18:02:38 to 2017-12-25 21:30:00
Data columns (total 5 columns):
home.alias             65 non-null object
scoring.home_points    65 non-null float64
away.alias             65 non-null object
scoring.away_points    65 non-null float64
winner                 65 non-null object
dtypes: float64(2), object(3)
memory usage: 3.0+ KB


In [86]:
hou_home.head()

,home.alias,scoring.home_points,away.alias,scoring.away_points,winner
scheduled,,,,,
2010-01-03 18:02:38,HOU,34.0,NE,27.0,HOU
2010-09-12 17:02:50,HOU,34.0,IND,24.0,HOU
2010-09-26 17:03:12,HOU,13.0,DAL,27.0,DAL
2010-10-10 17:03:05,HOU,10.0,NYG,34.0,NYG
2010-10-17 17:03:31,HOU,35.0,KC,31.0,HOU




- dataframe displays only date range from 2010-2017
- only where houston plays at home

## Save clean  selected data to  to csv

In [87]:
ls sports_data_clean/

astros_08_17_clean.csv*       nfl10clean.csv*        tsu_08_17_clean.csv*
dynamo_08_17_clean.csv*       nfl10yearsfull.csv*    uh_08_17_clean.csv*
hou_rockets_08_17_clean.csv*  nfl17years_clean.csv*
nfl_08_17_clean.csv*          rice_08_17_clean.csv*


In [88]:
hou_home.to_csv('sports_data_clean/hou_home_10_17_clean.csv')

In [89]:
## done